## LLM의 챗 템플릿


### 1. 토크나이저
모든 LLM들은 고유한 토크나이저가 존재합니다. 우선 transformers 패키지로 부터 AutoTokenizer를 임포트 합니다.

In [ ]:
from transformers import AutoTokenizer

AutoTokenizer.from_pretrained('모델 이름')을 사용하여 LLM 고유의 토크나이저를 불러
올 수 있습니다. 저희가 이번에 사용할 모델은 한국 회사 올거나이즈가 라마 (LLama) 모델을 한국어로 추가 학습한 모델인 allganize/Llama-3-Alpha-Ko-8B-Instruct입니다

In [ ]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

토크나이저의 메소드 encode()는 사람의 입력을 LLM이 이해할 수 있는 정수로 변환하고 decode()는 다시 이를 사람의 입력으로 해석하게 해줍니다.

In [ ]:
# 토킄화 과정을 내부적으로 수행 후에 바로 인코딩 한 결과
tokenizer.encode('안녕하세요. 반갑습니다.')

[128000, 101193, 124409, 13, 64857, 115072, 39331, 13]

위의 결과는’ 안녕하세요. 반갑습니다.’ 를 토크나이저가 토큰화 후에 정수 인코딩을 수행한 후의 결과입니다. 이를 다시 decode()의 입력으로 사용하여 복원해봅시다.


In [ ]:
tokenizer.decode([128000, 101193, 124409, 13, 64857, 115072, 39331, 13])

'<|begin_of_text|>안녕하세요. 반갑습니다.'

다시 ‘안녕하세요. 반갑습니다.’ 문자열로 복원된 것을 볼 수 있습니다. <|begin_of_text|>가 부착
되었지만, 이는 해당 모델의 특징이므로 여기서는 신경쓰지 않아도 됩니다.

###2. 챗 템플릿(Chat Template)
템플릿은 해당 모델이 만들어졌을 당시의 모델의 입력으로 넣을 때 반드시 지켜주어야 할 입력 형
식입니다. 이는 모델마다 다르며 이미 학습된 LLM 을 사용할 때 해당 LLM 의 챗 템플릿을 따르지 않
으면 제대로 된 성능을 얻기 어렵습니다. 모델 사용 전 반드시 기억해야 합니다. 이번 실습에서는
“allganize/Llama‑3‑Alpha‑Ko‑8B‑Instruct” 모델의 챗 템플릿에 대해서 알아볼 것입니다

해당 모델의 챗 템플릿을 알아보기 전에 우리가 해당 모델의 입력으로 넣을 시스템 프롬프트와 유저 프롬프트를 오픈 AI 형식으로 작성해보겠습니다. 오픈 AI 형식이란, 오픈 AI 의 GPT‑4 API 를 사용할 때의 형식을 의미합니다. 앞서 GPT‑4 API 를 사용하며 시스템 프롬프트와 유저 프롬프트에 대해서 설명한 바 있습니다
- 시스템 프롬프트: 모델의 역할이나 모델이 앞으로 해야할 일을 적는 구간.
- 유저 프롬프트: 모델에게 지금 요청할 것 (사용자의 질문)

임의로 아래와 같이 시스템 프롬프트와 유저 프롬프트를 GPT‑4 API 를 사용할 당시의 형식으로 작성해보
았습니다.

In [ ]:
messages = [
    {"role": "system", "content": "당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요"},
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]

모델에게 입력할 시스템 프롬프트와 유저 프롬프트를 오픈 AI 형식으로 작성하였고, 사용할 모델의토크나이저가 로드된 상황이라면 챗 템플릿 형식으로 빠르게 적용할 수 있습니다. tokenizer.
apply_chat_template()을 사용하되 입력으로 오픈 AI 형식으로 작성된 프롬프트를 넣으면 해당
LLM 이 사용하고 있는 챗 템플릿으로 자동 변환해줍니다.


In [ ]:
# tokenizer = False는 인코딩은 안하고 챗 템플릿만 적용
template_messages = tokenizer.apply_chat_template(messages, tokenize=False)
print(template_messages)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|>


위의 결과를 보면 시스템 프롬프트는 <|begin_of_text|><|start_header_id|>system<|
end_header_id|>와 <|eot_id|> 사이에 작성되고, 유저 프롬프트는 <|start_header_id
|>user<|end_header_id|>와 <|eot_id|> 사이에 작성 되어져 있는데 해당 형태가
“allganize/Llama‑3‑Alpha‑Ko‑8B‑Instruct” 모델의 챗 템플릿입니다. (구체적으로는 LLama‑3 의 챗 템플릿입니다.) 해당 모델을 사용할 때는 항상 시스템 프롬프트와 유저 프롬프트가 위의 형태를 가지도록 하여 모델의 입력으로 넣는 것이 좋습니다. 물론 모델에 넣을 때는 위 형식에서 정수 인코딩까지 되어져 있어야만 합니다. tokenizer=True를 사용하면 템플릿 적용 및 정수 인코딩 후의 결과를 볼 수 있습니다. 하지만 tokenizer=True가 기본 값이므로 tokenizer 인자의 값을 기재하지 않으면 됩니다.

In [ ]:
encodeds = tokenizer.apply_chat_template(messages)
print('템플릿 적용 및 정수 인코딩 후:\n', encodeds)

템플릿 적용 및 정수 인코딩 후:
 [128000, 128006, 9125, 128007, 271, 65895, 83628, 34804, 59777, 103896, 67119, 101139, 30426, 25941, 95252, 29726, 80052, 13, 100570, 119, 16969, 101264, 19954, 108280, 104834, 101360, 127923, 102893, 111964, 92245, 128009, 128006, 882, 128007, 271, 34804, 101066, 21028, 111902, 104193, 111833, 122115, 114942, 34983, 59269, 246, 128009]


실제로 위 결과를 다시 decode()하면 이미 챗 템플릿이 적용된 상태임을 확인할 수 있습니다.

In [ ]:
print('템플릿 적용 및 정수 인코딩 결과를 복원:\n', tokenizer.decode(encodeds))

템플릿 적용 및 정수 인코딩 결과를 복원:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|>


###3. 생성 프롬프트(Generation Prompt)
챗 템플릿을 적용할 때 추가로 고려해야 할 것으로 생성 프롬프트 (Generation Prompt) 라는 것이 있습니다. LLM 이 적절하게 응답을 할 수 있도록 도와주는 프롬프트입니다. apply_chat_template()을
사용할 때 add_generation_prompt=True를 사용하면 해당 프롬프트를 확인할 수 있습니다.

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
print(encodeds)

[128000, 128006, 9125, 128007, 271, 65895, 83628, 34804, 59777, 103896, 67119, 101139, 30426, 25941, 95252, 29726, 80052, 13, 100570, 119, 16969, 101264, 19954, 108280, 104834, 101360, 127923, 102893, 111964, 92245, 128009, 128006, 882, 128007, 271, 34804, 101066, 21028, 111902, 104193, 111833, 122115, 114942, 34983, 59269, 246, 128009, 128006, 78191, 128007, 271]


정수 인코딩이 된 결과가 나와서 add_generation_prompt=True를 추가하므로서 현재 챗 템플릿에 어떤 변화가 일어났는지 위 결과만으로는 확인이 어렵습니다. 위 결과를 decode()를 통해 정수 시퀀스를 텍스트 형태로 변환하여 챗 템플릿에 어떤 변화가 있었는지 보다 쉽게 확인해보겠습니다.


In [ ]:
print('템프릿 적용 및 정수 인코딩 결과를 복원:\n', tokenizer.decode((encodeds)))

템프릿 적용 및 정수 인코딩 결과를 복원:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>




위 형식의 챗 템플릿으로 학습된 LLM 사용 시, LLM 의 입력으로 사용하고 싶은 시스템 프롬프트와 유저 프롬프트가 있다면 아래와 같이 학습된 챗 템플릿 형식에 맞춰 배치해야 합니다. 예를 들어서 시스템 프롬프트로’ 당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요.’ 를 사용하고 유저 프롬프트로’ 은행의 기준 금리에 대해서 설명해줘’ 를 사용하여 LLM 의 답변을 얻고자 한다면 아래와 같이 입력을 넣어야 합니다

In [ ]:
"""system: 당신은 인공지능 어시스턴트입니다. 묻는말에 친절하고 정확하게 답변하세요.
:end
user: 은행의 기준 금리에 대해서 설명해줘 :end
assistant:
"""

'system: 당신은 인공지능 어시스턴트입니다. 묻는말에 친절하고 정확하게 답변하세요.\n:end\nuser: 은행의 기준 금리에 대해서 설명해줘 :end\nassistant:\n'

LLM 은 학습 당시에 assistant: 다음부터 실제 답변을 작성하도록 학습이 되어져 있기 때문에 LLM 에 위와 같은 입력을 넣어주어야 하는 것입니다. 위의 입력을 넣은 LLM 은 아래와 같이 답변을 생성할 것입니다. 입력의 챗 템플릿의 끝에 이미 assistant:가 붙어있기 때문에 이어서 답변만 작성하면 됩니다.

In [ ]:
"""<system> 너는 사용자의 질문에 친절하게 답변해야해. </system>
<user> 안녕 </user>
<assistant> 반갑습니다. <assistant>"""

'<system> 너는 사용자의 질문에 친절하게 답변해야해. </system>\n<user> 안녕 </user>\n<assistant> 반갑습니다. <assistant>'

위 형식의 챗 템플릿으로 학습된 LLM 을 사용한다고 가정하였을 때, 시스템 프롬프트와 유저 프롬프트를 새로 입력한다면 아래와 같이 학습된 챗 템플릿 형식에 맞춰 배치해야 합니다.

In [ ]:
"""<system> 당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요.
</system>
<user> 은행의 기준 금리에 대해서 설명해줘 </user>
<assistant>
"""

'<system> 당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요.\n</system>\n<user> 은행의 기준 금리에 대해서 설명해줘 </user>\n<assistant>\n'

LLM 은 학습 당시에 <<assistant>assistant> 다음부터 실제 답변을 작성하도록 학습이 되어져 있기 때문에 위와
같이 <<assistant>assistant>까지 부착하여 입력을 넣어주어야 하는 것입니다. 위의 입력을 넣은 LLM 은 아래와
같이 답변을 생성할 것입니다. 입력으로 사용된 챗 템플릿의 끝에 이미 <<assistant>assistant>가 붙어있기 때문
에 바로 답변만 작성하면 됩니다.


In [ ]:
"""중앙은행이 정하는 기준이 되는 금리로, 시중 금리와 경제 전반에 영향을 미칩니다. </assistant>"""

'중앙은행이 정하는 기준이 되는 금리로, 시중 금리와 경제 전반에 영향을 미칩니다. </assistant>'

방금 설명한 두 가지 예시에서는 생성 프롬프트가 각각 assistant:와 <<assistant>assistant>에 해당됩니다. 다시 말해 생성 프롬프트란 LLM 이 훈련된 챗 템플릿 형식에서 응답을 시작해야 하는 위치를 표시하는 부분입니다. “allganize/Llama‑3‑Alpha‑Ko‑8B‑Instruct” 의 <|start_header_id|>assistant
<|end_header_id|>도 마찬가지로 생성 프롬프트 역할을 하며, 입력 텍스트에 생성 프롬프트를 추
가한 후 LLM 에게 입력하면 모델이 어디서부터 자신의 답변을 생성해야 하는지 알려주는 역할을 합니다.
apply_chat_template()을 사용할 때 add_generation_prompt=True 옵션을 사용하면 각
모델에 맞는 생성 프롬프트가 자동으로 추가되어 LLM 이 답변 시 학습 시의 성능을 보존한 채 올바른 응답생성이 가능해집니다.
지금까지 LLM 에게 사용자의 입력을 전달할 때 반드시 알아야 할 개념인 챗 템플릿과 생성 프롬프트에 대해서 정리해보았습니다. GPT‑4 API 와 같은 API 서비스에서는 오픈 AI 형식과 같이 특정 형식으로 전달하면 내부적으로 챗 템플릿이 적용되어 LLM 이 호출되므로 API 사용자가 크게 신경 쓸 필요가 없지만, 모델을 다운받아서 직접 사용하는 경우에는 이러한 챗 템플릿을 적절히 적용하여 입력을 전달해야 합니다. 챗템플릿이나 생성 프롬프트를 올바르게 적용하지 않으면 다운로드하여 사용 중인 모델의 답변 성능이 크
게 저하될 수 있습니다. 또는 파인 튜닝을 한다고 하더라도 챗 템플릿을 무시하고 파인 튜닝을 한 모델은 본래 LLM 의 성능을 제대로 사용할 수 없습니다. 따라서 실제 운영 환경에서 모델을 다운받아서 활용할 때는 반드시 해당 모델의 챗 템플릿과 생성 프롬프트를 정확히 적용해야 할 것입니다.
